In [ ]:
import json
import os
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Dict, Any, Callable
import inspect
import textwrap
import glob
from horgues3.betting import format_betting_results
import math


# 日本語フォントの設定
plt.rcParams['font.family'] = 'UDEV Gothic NFLG'

In [ ]:
output_dir = 'outputs/training_20250606_184044'

In [ ]:
def plot_history_trend(history: List[Dict[str, Any]], 
                      value_extractors: List[str]):
    """
    historyリストから指定した位置の値を文字列式で抽出し、その推移をグラフ化する
    
    Args:
        history: 履歴データのリスト
        value_extractors: 各履歴項目から値を抽出する文字列式
                         単一の文字列（例: "x['loss']"）または
                         文字列のリスト（例: ["x['loss']", "x['accuracy']"]）
    """
    # 単一の文字列が渡された場合はリストに変換
    if isinstance(value_extractors, str):
        value_extractors = [value_extractors]
    
    # 抽出する値の数に応じて最適な行数と列数を計算
    num_plots = len(value_extractors)
    
    # 正方形に近い配置を計算
    cols = math.ceil(math.sqrt(num_plots))
    rows = math.ceil(num_plots / cols)
    
    # 各グラフを正方形に近い形にするため、figsize を調整
    fig_width = cols * 3  # 各グラフの幅を5インチに設定
    fig_height = rows * 3  # 各グラフの高さを5インチに設定
    
    fig, axes = plt.subplots(rows, cols, figsize=(fig_width, fig_height))
    
    # axesを1次元配列に変換（統一的な処理のため）
    axes = axes.flatten() if hasattr(axes, 'flatten') else [axes]
    
    # 各value_extractorに対してグラフを作成
    for idx, value_extractor in enumerate(value_extractors):
        # 文字列式をlambda関数に変換
        extractor_func = eval(f"lambda x: {value_extractor}")
        
        # lambda関数を使って値を抽出
        values = []
        epochs = []
        
        for i, hist_item in enumerate(history):
            value = extractor_func(hist_item)
            values.append(value)
            epochs.append(i + 1)  # エポック番号は1から始める

        # タイトルを処理
        wrapped_title = textwrap.fill(value_extractor, width=30)

        # グラフを作成
        ax = axes[idx]
        ax.plot(epochs, values, linewidth=2)
        ax.set_title(wrapped_title, fontweight='bold')
        ax.set_xlabel('epoch', fontweight='bold')
        ax.set_ylabel('value', fontweight='bold')
        ax.grid(True, alpha=0.3)
        
        # 正方形に近い形にするためアスペクト比を調整
        ax.set_aspect('auto')
    
    # 余ったサブプロットを非表示にする
    for idx in range(num_plots, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()

In [ ]:
history_path = os.path.join(output_dir, 'training_history.json')
with open(history_path) as f:
    history = json.load(f)

plot_history_trend(history, [
    "x['train']['loss']",
    "x['val']['loss']",
    "x['val']['accuracy']['tansho']['hit_rate']",
    "x['val']['accuracy']['fukusho']['hit_rate']",
    "x['val']['accuracy']['umaren']['hit_rate']",
    "x['val']['accuracy']['wide']['hit_rate']",
    "x['val']['accuracy']['umatan']['hit_rate']",
    "x['val']['expected_value_betting']['tansho']['profit_rate']",
    "x['val']['expected_value_betting']['fukusho']['profit_rate']",
    "x['val']['expected_value_betting']['umaren']['profit_rate']",
    "x['val']['expected_value_betting']['wide']['profit_rate']",
    "x['val']['expected_value_betting']['umatan']['profit_rate']",
])

In [ ]:
probabilities_path = glob.glob(os.path.join(output_dir, 'validation_probabilities_epoch_*.npz'))[-1]
with np.load(probabilities_path, allow_pickle=True) as data:
    race_ids = data['race_ids']
    probabilities = {key: data[key] for key in data.keys() if key != 'race_ids'}

# 中央競馬以外を除外
valid_indices = []
valid_race_ids = []
for i, race_id in enumerate(race_ids):
    track_code = race_id[8:10]
    if '01' <= track_code <= '10':
        valid_indices.append(i)
        valid_race_ids.append(race_id)
valid_probabilities = {}
for key in probabilities.keys():
    valid_probabilities[key] = probabilities[key][valid_indices]       

formatted_results = format_betting_results(valid_race_ids, valid_probabilities)
print(formatted_results)